In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
%cd /content/drive/MyDrive/Project 

/content/drive/MyDrive/Project


In [35]:
module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


In [36]:
! pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50V2, VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input, MaxPool2D, BatchNormalization, Dropout, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

In [38]:
# Creating dataframe with art movement class weights
art_movements = pd.DataFrame()
art_movements['names'] = ['Academic', 'Baroque', 'Expressionism', 'Japanese', 'Neoclassicism', 'Nouveau', 'Primitivism', 'Realism', 'Renaissance', 'Rococo','Romanticism', 'Symbolism', 'Western Medieval']
art_movements['# of paintings'] = [500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500]
art_movements = art_movements.sort_values(by=['# of paintings'], ascending=False)
art_movements = art_movements.reset_index(drop=True)
art_movements['class_weight'] = art_movements['# of paintings'].sum() / (art_movements.shape[0] * art_movements['# of paintings'])
class_weights = art_movements['class_weight'].to_dict()
print(class_weights)
art_movements_name = art_movements['names'].values
print(art_movements_name)

{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0}
['Academic' 'Baroque' 'Expressionism' 'Japanese' 'Neoclassicism' 'Nouveau'
 'Primitivism' 'Realism' 'Renaissance' 'Rococo' 'Romanticism' 'Symbolism'
 'Western Medieval']


In [39]:
batch_size = 64
train_input_shape = (256, 256, 3)

datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range= 5,
    zoom_range=0.05,
    horizontal_flip=True,
    validation_split=0.1
)

train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Project/new_dataset',
    target_size=train_input_shape[0:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle =True,
    subset='training',
    seed=123,
    classes=art_movements_name.tolist()
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Project/new_dataset', #use the same data with train
    target_size=train_input_shape[0:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle =True,
    subset='validation',
    seed=123,
    classes=art_movements_name.tolist()
)

Found 1200 images belonging to 13 classes.
Found 133 images belonging to 13 classes.


In [40]:
# Adding model steps
Step_Train = train_generator.n//train_generator.batch_size
Step_Validation = validation_generator.n//validation_generator.batch_size
print("Total number of batches =", Step_Train, "and", Step_Validation)

Total number of batches = 18 and 2


In [50]:
# Convolution Neural Networks (CNN)
model = Sequential()

model.add(Conv2D(8, kernel_size=(3,3), padding='same', input_shape = (256,256,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(3, 3)))

model.add(Conv2D(16, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(13, activation='softmax'))
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 256, 256, 8)       224       
                                                                 
 activation (Activation)     (None, 256, 256, 8)       0         
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 85, 85, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 85, 85, 16)        1168      
                                                                 
 batch_normalization (BatchN  (None, 85, 85, 16)       64        
 ormalization)                                                   
                                                                 
 activation_1 (Activation)   (None, 85, 85, 16)       

In [51]:
model.fit(train_generator, 
          validation_data=validation_generator, 
          epochs=20)

Epoch 1/20
19/19 [==============================] - 86s 4s/step - loss: 3.3220 - accuracy: 0.0900 - val_loss: 2.5654 - val_accuracy: 0.0677
Epoch 2/20
19/19 [==============================] - 79s 4s/step - loss: 3.3013 - accuracy: 0.0808 - val_loss: 2.5725 - val_accuracy: 0.0752
Epoch 3/20
19/19 [==============================] - 73s 4s/step - loss: 3.3221 - accuracy: 0.0792 - val_loss: 2.5837 - val_accuracy: 0.0752
Epoch 4/20
19/19 [==============================] - 76s 4s/step - loss: 3.2866 - accuracy: 0.0858 - val_loss: 2.5901 - val_accuracy: 0.0752
Epoch 5/20
19/19 [==============================] - 77s 4s/step - loss: 3.2099 - accuracy: 0.0867 - val_loss: 2.5993 - val_accuracy: 0.0752
Epoch 6/20
19/19 [==============================] - 72s 4s/step - loss: 3.1765 - accuracy: 0.0892 - val_loss: 2.6094 - val_accuracy: 0.0752
Epoch 7/20
19/19 [==============================] - 72s 4s/step - loss: 3.1750 - accuracy: 0.0875 - val_loss: 2.6121 - val_accuracy: 0.0752
Epoch 8/20
19/19 [==

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  #Upload image
  path = fn
  img = load_img(path, target_size=(256, 256))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  #Predict image
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

In [48]:
if(np.argmax(classes[0])) == 12:
  print("Western Medieval: Artworks belonging to Byzantine, Pre-Romanesque, Gothic, Medieval, and Coptic movements")
elif(np.argmax(classes[0])) == 11:
  print("Symbolism: Symbolism was a late nineteenth-century art movement of French, Russian and Belgian origin in poetry and other arts. Distinct from, but related to, the style of literature, symbolism in art is related to the gothic component of Romanticism and Impressionism.")
elif(np.argmax(classes[0])) == 10:
  print("Romanticism: Romanticism (also the Romantic era or the Romantic period) was an artistic, literary, musical and intellectual movement that originated in Europe toward the end of the 18th century and in most areas was at its peak in the approximate period from 1800 to 1850. Romanticism was characterized by its emphasis on emotion and individualism as well as glorification of all the past and nature, preferring the medieval rather than the classical.")
elif(np.argmax(classes[0])) == 9:
  print("Rococo: Rococo infused the world of art and interior design with an aristocratic idealism that favored elaborate ornamentation and intricate detailing. The paintings that became the signature to the era were created in celebration of Rococo's grandiose ideals and lust for the aristocratic lifestyle and pastimes. The movement, which developed in France in the early 1700s, evolved into a new, over-the-top marriage of the decorative and fine arts, which became a visual lexicon that infiltrated 18th-century continental Europe.")
elif(np.argmax(classes[0])) == 8:
  print("Renaissance (Western): The Renaissance is a period in European history marking the transition from the Middle Ages to modernity and covering the 15th and 16th centuries, characterized by an effort to revive and surpass ideas and achievements of classical antiquity.")
elif(np.argmax(classes[0])) == 7:
  print("Realism: Realism in the arts is the attempt to represent subject matter truthfully, without artificiality and avoiding artistic conventions, implausible, exotic, and supernatural elements. Realism has been prevalent in the arts for many periods and is in large part a matter of technique and training, and the avoidance of stylization.")
elif(np.argmax(classes[0])) == 6:
  print("Primitivism (Naïve Art): Primitivism is a Western art movement that borrows visual forms from non-Western or prehistoric peoples, such as Paul Gauguin's inclusion of Tahitian motifs in paintings and ceramics. Borrowings from primitive art have been important to the development of modern art.")
elif(np.argmax(classes[0])) == 5:
  print("Art Nouveau: Art Nouveau is an international style of art, architecture, and applied art, especially the decorative arts, that was most popular between 1890 and 1910. A reaction to the academic art of the 19th century, it was inspired by natural forms and structures, particularly the curved lines of plants and flowers.")
elif(np.argmax(classes[0])) == 4:
  print("Neoclassicism: Europeans of the 18th century Age of Enlightenment wanted their artwork and architecture to mirror and carry the same standards as the idealized works of the Greeks and Romans. In conjunction with the exciting archaeological rediscoveries of Pompeii and Herculaneum in Rome, Neoclassicism arose as artists and architects infused their work with past Greco-Roman ideals. A return to the study of science, history, mathematics, and anatomical correctness abounded, replacing the Rococo vanity culture and court-painting climate that preceded.")
elif(np.argmax(classes[0])) == 3:
  print("Japanese Art: Art from Japan from the Muromachi period (1392-1573) all the way to the Shōwa period (1926-1989)")
elif(np.argmax(classes[0])) == 2:
  print("Expressionism: Expressionism was a modernist movement, initially in poetry and painting, originating in Germany at the beginning of the 20th century. Its typical trait is to present the world solely from a subjective perspective, distorting it radically for emotional effect in order to evoke moods or ideas. Expressionist artists sought to express the meaning of emotional experience rather than physical reality.")
elif(np.argmax(classes[0])) == 1:
  print("Baroque: The origin of the term Baroque is a bit ambiguous. Many scholars think it was derived from the Portuguese barrocco, meaning an imperfect or irregularly shaped pearl. And some, like the philosopher Jean-Jacques Rousseau thought it was derived from the Italian barocco, a term used to describe an obstacle in formal logic in the medieval period.")
else:
  print("Academic Art: Academic art, or Academicism, is a style of painting and sculpture produced under the influence of European academies of art. Specifically, academic art is the art and artists influenced by the standards of the French Académie des Beaux-Arts, which practiced under the movements of Neoclassicism and Romanticism, and the art that followed these two movements in the attempt to synthesize both of their styles.")

Primitivism (Naïve Art): Primitivism is a Western art movement that borrows visual forms from non-Western or prehistoric peoples, such as Paul Gauguin's inclusion of Tahitian motifs in paintings and ceramics. Borrowings from primitive art have been important to the development of modern art.


# **NEW**